In [8]:
#使用神经网络
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
## 基础工具
import numpy as np
import pandas as pd
import warnings
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import jn
from IPython.display import display, clear_output
import time
import missingno as msno
#导入warnings包，利用过滤器来实现忽略警告语句。
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [10]:
#载入训练集和测试集；
path = './2sc/'
train_data = pd.read_csv(path+'used_car_train_20200313.csv', sep=' ')
test_data = pd.read_csv(path+'used_car_testB_20200421.csv', sep=' ')

In [11]:
train_data["notRepairedDamage"].replace("-", "0.0", inplace=True)
train_data["notRepairedDamage"] = train_data["notRepairedDamage"].astype("float64")

In [12]:
test_data["notRepairedDamage"].replace("-", "0.0", inplace=True)
test_data["notRepairedDamage"] = test_data["notRepairedDamage"].astype("float64")

In [13]:
train_data.fillna(train_data.median(), inplace=True)
test_data.fillna(test_data.median(), inplace=True)

In [14]:
train_data["power"].describe()

count    150000.000000
mean        119.316547
std         177.168419
min           0.000000
25%          75.000000
50%         110.000000
75%         150.000000
max       19312.000000
Name: power, dtype: float64

In [15]:
test_data["power"][test_data["power"]>600] = 600

In [16]:
#特征归一化
train_data["power"][train_data["power"]>600] = 600
features = train_data.columns.tolist()
features.remove("price")
features.remove("SaleID")
features

['name',
 'regDate',
 'model',
 'brand',
 'bodyType',
 'fuelType',
 'gearbox',
 'power',
 'kilometer',
 'notRepairedDamage',
 'regionCode',
 'seller',
 'offerType',
 'creatDate',
 'v_0',
 'v_1',
 'v_2',
 'v_3',
 'v_4',
 'v_5',
 'v_6',
 'v_7',
 'v_8',
 'v_9',
 'v_10',
 'v_11',
 'v_12',
 'v_13',
 'v_14']

In [17]:
min_max_scaler = MinMaxScaler()
x = min_max_scaler.fit_transform(train_data[features].values)
x_test = min_max_scaler.transform(test_data[features].values)

In [18]:
y = train_data["price"].values

In [20]:
from sklearn.model_selection import GridSearchCV,cross_val_score,StratifiedKFold,train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2)

In [21]:
#搭建模型
model = keras.Sequential([
    keras.layers.Dense(300, activation="relu", input_shape=[len(features)]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(200, activation="relu"),
    keras.layers.Dense(1)
])
model.compile(loss="mae", optimizer="Adam")


Epoch 1/200
118/118 [==============================] - 4s 38ms/step - loss: 4861.4790
Epoch 2/200
118/118 [==============================] - 5s 39ms/step - loss: 2717.2000
Epoch 3/200
118/118 [==============================] - 5s 39ms/step - loss: 1279.7288
Epoch 4/200
118/118 [==============================] - 5s 39ms/step - loss: 1081.4524
Epoch 5/200
118/118 [==============================] - 4s 38ms/step - loss: 1001.5699
Epoch 6/200
118/118 [==============================] - 5s 43ms/step - loss: 941.6675
Epoch 7/200
118/118 [==============================] - 5s 41ms/step - loss: 894.5315
Epoch 8/200
118/118 [==============================] - 5s 40ms/step - loss: 856.5559
Epoch 9/200
118/118 [==============================] - 6s 47ms/step - loss: 825.1848
Epoch 10/200
118/118 [==============================] - 5s 45ms/step - loss: 798.6883
Epoch 11/200
118/118 [==============================] - 5s 41ms/step - loss: 775.3813
Epoch 12/200
118/118 [==============================] - 5s

In [ ]:
#模型评估
print("训练集评估 MAE： ", mean_absolute_error(train_y, model.predict(train_x)))
print("测试集评估 MAE： ", mean_absolute_error(test_y, model.predict(test_x)))

In [ ]:
#用全量数据训练
model = keras.Sequential([
    keras.layers.Dense(200, activation="relu", input_shape=[len(features)]),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(1)
])
model.compile(loss="mae", optimizer="Adam")
model.fit(x, y, batch_size=1024, epochs=100)

In [ ]:
#输出结果
predict_y = model.predict(x_test)

result = pd.DataFrame()
result["SaleID"] = test_data["SaleID"]
result["price"] = predict_y
#result["price"][result["price"] < 11] = 11
result.loc[result["price"] <11, "price"] = 11
result.loc[result["price"] >99999, "price"] = 99999
result.to_csv("ans_nn.csv", index=False)

In [ ]:
show_stats(predict_y)

In [ ]:
show_stats(result)